In [86]:
# https://huggingface.co/docs/peft/index

import os 
from transformers import GPT2Tokenizer, GPT2LMHeadModel

### Utils
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
### 

math_example_path = os.getcwd() + '/data/What_Is_Mathematics_An_Elementary_Approach_to_Ideas_and_Methods.txt'

with open(math_example_path, "r",  encoding="utf8") as f:
     math_example_text = f.read()

math_example_text = math_example_text.replace("\n", "")
math_sentences = math_example_text.split(".")

In [87]:
print(len(math_sentences))
math_sentences = [item for item in math_sentences if 50 < len(item) < 100]
print(math_sentences[1])

9421
 Today the traditional place of mathematics in education is in grave danger


In [88]:
math_sentences[53]

' These laws of arithmetic are very simple, and may seem obvious'

In [89]:
output_dir = "./finetuned_llm_all"

model_name = "gpt2" #output_dir #
tokenizer = GPT2Tokenizer.from_pretrained(model_name) #gpt2
llm_model = GPT2LMHeadModel.from_pretrained(model_name) # gpt2
device = torch.device('cuda')
llm_model.to(device)
tokenizer.pad_token = tokenizer.eos_token

# Configure the model
llm_model.config.pad_token_id = tokenizer.pad_token_id
llm_model.config.eos_token_id = tokenizer.eos_token_id
llm_model.config.vocab_size = llm_model.config.vocab_size + len(tokenizer.get_added_vocab())
llm_model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [90]:
#len_params = len(list(llm_model.parameters()))
#for i, param in enumerate(llm_model.parameters()):
#    param.requires_grad = i/len_params > 0.8 # train last 20% of layers

In [91]:
encoded_data = tokenizer.batch_encode_plus(math_sentences, add_special_tokens=False, return_tensors='pt', padding=True).to(device)
BATH_SIZE = 10

batch_data = list(chunks(encoded_data["input_ids"], BATH_SIZE))

In [97]:
import numpy as np
tst_text = np.random.choice(math_sentences, 30)
tst_data = tokenizer.batch_encode_plus(tst_text, add_special_tokens=True, return_tensors='pt', padding=True).to(device)
output = llm_model.generate(tst_data["input_ids"])

for i, text in enumerate(tst_text):
    print("input:", text)
    print("output", tokenizer.decode(output[i]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


input:  On the right side, the term in brackets is likewise an integer
output  On the right side, the term in brackets is likewise an integer<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
input:  The radius of the circle is thus divided into two segments, x and 1 — x
output  The radius of the circle is thus divided into two segments, x and 1 — x<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

C:\Users\tempdelta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\utils.py:1136: UserWarning: Input length of input_ids is 47, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [93]:
output = llm_model.generate(tokenizer.encode(math_sentences[1], add_special_tokens=False, return_tensors='pt', padding=False).to(device))
print(math_sentences[1])
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Today the traditional place of mathematics in education is in grave danger
 Today the traditional place of mathematics in education is in grave danger.

The problem is that the


In [94]:
import torch

optimizer = torch.optim.AdamW(llm_model.parameters(), lr=1e-5)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [96]:
import mlflow

EPOCHS=10

mlflow.end_run()
gradient_accumulation_steps = 10
grad_counter = 0

with mlflow.start_run(run_name='final') as run:

    for epoch in range(EPOCHS):
        for batch in batch_data: 
            outputs = llm_model(batch, labels=batch)
            loss = outputs.loss
            loss.backward()

            if grad_counter % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

                print("batch loss ", loss.item())

                
            grad_counter += 1
                            

        # log metric every epoch
        mlflow.log_metric('loss', loss.item())
        mlflow.log_metric('lr', get_lr(optimizer))

        
        output = llm_model.generate(tst_data["input_ids"].to(device))
        test_output = ""
        for i in range(output.shape[0]):
            tmp_str = tokenizer.decode(output[i])
            test_output += "\n" + tmp_str
        mlflow.log_text(test_output, f"test_{epoch}.txt")
        #### 

        
        # Save the fine-tuned model every epoch 
        print("MODEL saved loss ", loss)
        llm_model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

batch loss  1.2422912120819092
batch loss  1.3767868280410767
batch loss  1.6074696779251099
batch loss  1.457045555114746
batch loss  1.4672168493270874
batch loss  1.6535619497299194


KeyboardInterrupt: 

In [85]:
output = llm_model.generate(tokenizer.encode("Do you love the war ", add_special_tokens=True, return_tensors='pt', padding=True).to(device), temperature=40)
print("vvv", tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


vvv Do you love the war?"

"I love the war?"

"I


In [27]:
from transformers import TextDataset

train_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=math_example_path,
      block_size=128)

C:\Users\tempdelta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [28]:
from transformers import (
    TrainingArguments, 
    Trainer, 
    default_data_collator, 
)
import mlflow 
mlflow.end_run()

training_args = TrainingArguments(output_dir='test_trainer', 
                                  #evaluation_strategy='epoch',
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  gradient_accumulation_steps=20, #
                                  num_train_epochs = 2,
                                  fp16=False,)

trainer = Trainer(
    model=llm_model,
    args=training_args,
    train_dataset=dataset,
    #eval_dataset=dataset,
    tokenizer=tokenizer,
    # Data collator will default to DataCollatorWithPadding, so we change it.
    data_collator=default_data_collator,
    compute_metrics=None,
    preprocess_logits_for_metrics=None,
)

trainer.train()

RuntimeError: Could not infer dtype of DatasetInfo

In [23]:
dataset

[Dataset({
     features: ['text'],
     num_rows: 31749
 })]